In [10]:
import types
import math
from typing import Callable

import torch
import torch.nn as nn
import torch.nn.functional as F

import timm

In [ ]:
dino = timm.create_model('vit_small_patch16_224_dino', pretrained=True)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name vit_small_patch16_224_dino to current vit_small_patch16_224.dino.
  model = create_fn(


model.safetensors:   0%|          | 0.00/86.7M [00:00<?, ?B/s]

In [39]:
patch_size = [16, 16]
hooks = [2, 5, 8, 11]
hook_patch = True
start_index = 1

In [17]:
pretrained = nn.Module()
pretrained.model = dino

In [51]:
activations = {}
def get_activation(name: str) -> Callable:
    def hook(model, inputs, outputs):
        activations[name] = outputs
    return hook

In [ ]:
for i in range(len(hooks)):
    pretrained.model.blocks[hooks[i]].register_forward_hook(get_activation(f'{i}')) # Get shape of 2nd, 5th, 6th... Block of ViT
if hook_patch: pretrained.model.pos_drop.register_forward_hook(get_activation('4'))

In [56]:
dino.eval()
output = dino(torch.rand(5, 3, 224, 224))
output.shape

torch.Size([5, 384])

In [69]:
for i in range(len(hooks)):
    print(f"Block {hooks[i]} Shape:", list(activations[f'{i}'].shape))
if hook_patch:
    print(f"Dropout Block Shape:", list(activations['4'].shape))

Block 2 Shape: [5, 197, 384]
Block 5 Shape: [5, 197, 384]
Block 8 Shape: [5, 197, 384]
Block 11 Shape: [5, 197, 384]
Dropout Block Shape: [5, 197, 384]


In [67]:
list(activations['4'].shape)

[5, 197, 384]

In [ ]:
# B = batch size
# N = number of tokens
# C = embedding dim

Dropout(p=0.0, inplace=False)

In [88]:
B = 5
N = 77
C = 768
start_index = 1

x = torch.rand(B, N, C) # [5, 77, 768]

# Every local patch token gains global context
if start_index == 2:
    readout = (x[:, 0] + x[:, 1]) / 2 # (CLS + DIST) / 2
else:
    readout = x[:, 0]                 # CLS

# readout: [B,             C]
# x      : [B, N - 2 or 1, C]
out = x[: , start_index:] + readout.unsqueeze(1)
out.shape

torch.Size([5, 76, 768])

In [96]:
class Readout(nn.Module):
    """
    Adds CLS and/or DIST Tokens to the patches
    """
    def __init__(self, start_index = 1):
        super().__init__()

        self.start_index = start_index
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # X [B, N, C]
        # Every local patch token gains global context
        if self.start_index == 2:
            readout = (x[:, 0] + x[:, 1]) / 2 # (CLS + DIST) / 2
        else:
            readout = x[:, 0]                 # CLS
        
        # readout: [B,             C]
        # x      : [B, N - 2 or 1, C]
        return x[: , self.start_index:] + readout.unsqueeze(1)

readout = Readout(start_index=1)
readout(torch.rand(5, 77, 768)).shape

torch.Size([5, 76, 768])

In [99]:
class Transpose(nn.Module):
    """
    from [B, N, C] to [B, C, N]
    """
    def __init__(self, dim1: int, dim2: int):
        super().__init__()
        self.dim1 = dim1
        self.dim2 = dim2
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return x.transpose(self.dim1, self.dim2).contiguous()

transpose = Transpose(1, 2)
transpose(torch.rand(5, 77, 768)).shape

torch.Size([5, 768, 77])

In [ ]:
x = torch.rand(5, 3, 224, 224)
a = pretrained.model.patch_embed.proj(x) # a: [B, embedding dimension, H // 16   W // 16]
a = a.flatten(2)                         # a: [B, embedding dimension, H // 16 x W // 16]
a = a.transpose(1,2)                     # a: [B, H // 16 x W // 16, embedding dimension]
x = a
# x: [B, patch_dim, embedding dimension]
# x: [B, N, C]



torch.Size([5, 196, 384])

In [101]:
types.MethodType

method